In [151]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import pipeline
import numpy as np
import evaluate

In [152]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [153]:
# Load the dataset with a specified encoding
file_path = '/Users/lukishyadav/Desktop/engineering/case_studies/ner_casestudy/data/ner_dataset.csv'  # Replace with your file path
data = pd.read_csv(file_path, encoding='ISO-8859-1')


In [154]:
# # Drop rows with NaN values
data = data.dropna(subset=['Word','POS','Tag'])

In [155]:
# Group the data by sentences
data['Sentence #'] = data['Sentence #'].ffill()  # Fill forward to propagate sentence IDs
sentences = data.groupby('Sentence #').apply(lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                                                      s['POS'].values.tolist(),
                                                                                      s['Tag'].values.tolist())])


In [156]:
# Convert the groupby object to a list of sentences
sentences = [s for s in sentences]


In [157]:
# Split the dataset into training, validation, and test sets (20% for test)
train_sentences, test_sentences = train_test_split(sentences, test_size=0.20, random_state=42)
train_sentences, val_sentences = train_test_split(train_sentences, test_size=0.25, random_state=42)  # 0.25 * 0.80 = 0.20


In [158]:
# Convert to Hugging Face Datasets format
def convert_to_dict(sentences):
    words = [[word for word, pos, tag in sentence] for sentence in sentences]
    pos_tags = [[pos for word, pos, tag in sentence] for sentence in sentences]
    ner_tags = [[tag for word, pos, tag in sentence] for sentence in sentences]
    return {"tokens": words, "pos_tags": pos_tags, "ner_tags": ner_tags}


In [159]:
train_data = convert_to_dict(train_sentences)
val_data = convert_to_dict(val_sentences)
test_data = convert_to_dict(test_sentences)


In [160]:
# Create a dataset dictionary
dataset_dict = DatasetDict({
    'train': Dataset.from_dict(train_data),
    'validation': Dataset.from_dict(val_data),
    'test': Dataset.from_dict(test_data)
})

In [161]:
# Define unique tags
unique_tags = list(set(tag for doc in dataset_dict['train']['ner_tags'] for tag in doc))
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [162]:
# Tokenizer
model_checkpoint = "bert-base-uncased"
model_checkpoint = "dslim/bert-base-NER"
model_checkpoint = "nlpso/m1_ind_layers_ref_ptrn_cmbert_iob2_level_2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/anaconda3/envs/gen_3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [163]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2id[label[word_idx]])
            else:
                label_ids.append(tag2id[label[word_idx]] if True else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [164]:
tokenized_datasets = dataset_dict.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/28769 [00:00<?, ? examples/s]

Map:   0%|          | 0/9590 [00:00<?, ? examples/s]

Map:   0%|          | 0/9590 [00:00<?, ? examples/s]

In [165]:
# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [166]:
# Model
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(unique_tags),ignore_mismatched_sizes=True)
#model.to(device)

# Adjust the model's classification layer to match the number of labels
model.classifier = torch.nn.Linear(model.classifier.in_features, len(unique_tags))

# Ensure the model is initialized properly
model.num_labels = len(unique_tags)


Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at nlpso/m1_ind_layers_ref_ptrn_cmbert_iob2_level_2 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21, 768]) in the checkpoint and torch.Size([17, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([21]) in the checkpoint and torch.Size([17]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [167]:
# Metrics
metric = evaluate.load("seqeval")

In [168]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [169]:
# Training arguments
args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    use_mps_device=True
)

In [170]:
# Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [171]:
# import os

# mlruns_dir = '/Users/lukishyadav/Desktop/engineering/case_studies/ner_casestudy/mlruns'
# if not os.path.exists(mlruns_dir):
#     os.makedirs(mlruns_dir)

In [172]:
%time
# Train the model
trainer.train()

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 8.34 µs


/opt/anaconda3/envs/gen_3.8/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Exception: '/Users/lukishyadav/Desktop/engineering/case_studies/ner_casestudy/mlruns' does not exist.

In [ ]:
# Evaluate the model
results = trainer.evaluate(tokenized_datasets["test"])
print(results)


  0%|          | 0/600 [00:00<?, ?it/s]

/opt/anaconda3/envs/gen_3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.3194218575954437, 'eval_precision': 0.6042061986084757, 'eval_recall': 0.5889697268635551, 'eval_f1': 0.5964906803209592, 'eval_accuracy': 0.9150174234841153, 'eval_runtime': 55.0558, 'eval_samples_per_second': 174.187, 'eval_steps_per_second': 10.898, 'epoch': 0.02}
{'eval_loss': 0.3194218575954437, 'eval_precision': 0.6042061986084757, 'eval_recall': 0.5889697268635551, 'eval_f1': 0.5964906803209592, 'eval_accuracy': 0.9150174234841153, 'eval_runtime': 55.0558, 'eval_samples_per_second': 174.187, 'eval_steps_per_second': 10.898, 'epoch': 0.02}


In [ ]:
# Check for MPS device availability
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Predict with the model
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple",device=device
                        #,device=0 if torch.cuda.is_available() else -1
                        )
sample_text = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge."
sample_text = "Hey, hello!"
ner_results = ner_pipeline(sample_text)
print(ner_results)

[{'entity_group': 'LABEL_4', 'score': 0.9469243, 'word': 'Hey, hello!', 'start': 0, 'end': 11}]


In [ ]:
ner_results

[{'entity_group': 'LABEL_6',
  'score': 0.8172186,
  'word': 'hugging face inc. is a company based in new york city. its headquarters are in dumbo, therefore very close to the manhattan bridge.',
  'start': 0,
  'end': 131}]

In [ ]:
train_data.keys()


dict_keys(['tokens', 'pos_tags', 'ner_tags'])

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'pos_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 28769
    })
    validation: Dataset({
        features: ['tokens', 'pos_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9590
    })
    test: Dataset({
        features: ['tokens', 'pos_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9590
    })
})